In [ ]:
! pip install -U accelerate
! pip install -U transformers
!pip install sacremoses datasets evaluate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:0

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, get_linear_schedule_with_warmup, set_seed, AutoModelForSequenceClassification
from tqdm import tqdm
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
torch.manual_seed(42)

from peft import (
    get_peft_model,
    LoraConfig,
    PeftType,
)

from datasets import load_dataset, Dataset,DatasetDict
import datasets
import evaluate
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device being used is: ", device)

#GPT-2 is a model with absolute position embeddings so it's usually advised to pad the inputs on the right rather than the left. Remember this
tokenizer = AutoTokenizer.from_pretrained("chaoyi-wu/PMC_LLAMA_7B")
tokenizer.padding_side = "left"

model = AutoModelForSequenceClassification.from_pretrained("chaoyi-wu/PMC_LLAMA_7B", num_labels=2, return_dict=True)
model.resize_token_embeddings(len(tokenizer))

#model = model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(28897, 2560)

In [ ]:
peft_type = PeftType.LORA
num_epochs = 15
batch_size = 4

# Define LoRA Config
peft_config  = LoraConfig(
 r=8,
 lora_alpha=16,
 inference_mode=False,
 lora_dropout=0.1,
 )
lr = 3e-4
#3e-4
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
#model

trainable params: 1,179,648 || all params: 347,944,960 || trainable%: 0.33903293210512375


In [ ]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Colab Notebooks/dissertation/data/final_clinicalText.csv")

dataset = dataset.rename_column("extracted_text", "Text")
dataset = dataset.rename_column("clinical_action", "labels")

dataset = dataset.class_encode_column('labels')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting to class labels:   0%|          | 0/188 [00:00<?, ? examples/s]

In [ ]:
train_testvalid = dataset['train'].train_test_split(test_size=0.4, seed=42, stratify_by_column="labels")
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42, stratify_by_column="labels")
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
# Convert to a pandas DataFrame
df_test = pd.DataFrame( dataset['test'] )
df_train = pd.DataFrame( dataset['train'] )
df_valid = pd.DataFrame( dataset['valid'] )

print(df_train.groupby("labels").size())
print(df_valid.groupby("labels").size())
print(df_test.groupby("labels").size())


# Convert back to dataset
dataset = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train),
        "test": Dataset.from_pandas(df_test),
        "valid": Dataset.from_pandas(df_valid),
    }
)

labels
0    70
1    42
dtype: int64
labels
0    24
1    14
dtype: int64
labels
0    24
1    14
dtype: int64


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["Text"])
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["Text"])

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [ ]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")
# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_dataset["valid"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)
test_dataloader = DataLoader(tokenized_dataset["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)

In [ ]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
metric = evaluate.load('bstrai/classification_report')
metric2 = evaluate.load('hyperml/balanced_accuracy')

In [ ]:
#rsults without fine tunning
model.to(device)
for step, batch in enumerate(tqdm(test_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )
    metric2.add_batch(
        predictions=predictions,
        references=references,
    )


eval_metric = metric.compute()
eval_metric2 = metric2.compute()

print()
print()
print("results on the test data, before fine tunning, letters, llama:")
print(eval_metric)
print(eval_metric2)

100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


results on the test data:
{'0': {'precision': 0.8888888888888888, 'recall': 0.3333333333333333, 'f1-score': 0.48484848484848486, 'support': 24}, '1': {'precision': 0.4482758620689655, 'recall': 0.9285714285714286, 'f1-score': 0.6046511627906977, 'support': 14}, 'accuracy': 0.5526315789473685, 'macro avg': {'precision': 0.6685823754789272, 'recall': 0.6309523809523809, 'f1-score': 0.5447498238195914, 'support': 38}, 'weighted avg': {'precision': 0.7265577737447066, 'recall': 0.5526315789473685, 'f1-score': 0.528986313564037, 'support': 38}}
{'balanced_accuracy': 0.6309523809523809}


In [ ]:
import evaluate
import copy


best_f1 = 0.0
best_epoch = -1
best_model = None

model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric2.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric2 = metric2.compute()
    print(f"epoch {epoch}:", eval_metric2)

    # Check if the current F1 score is better than the previous best
    if eval_metric2["balanced_accuracy"] > best_f1:
        best_f1 = eval_metric2["balanced_accuracy"]
        best_epoch = epoch
        print("new best model found, saving...")

        # Save the model checkpoint with the best F1 score
        best_model = copy.deepcopy(model)

100%|██████████| 2/2 [00:00<00:00,  3.84it/s]


epoch 0: {'balanced_accuracy': 0.6904761904761905}
new best model found, saving...


100%|██████████| 2/2 [00:00<00:00,  3.97it/s]


epoch 1: {'balanced_accuracy': 0.6755952380952381}


100%|██████████| 2/2 [00:00<00:00,  4.55it/s]


...

epoch 17: {'balanced_accuracy': 0.6755952380952381}


100%|██████████| 2/2 [00:00<00:00,  3.73it/s]


epoch 18: {'balanced_accuracy': 0.6755952380952381}


100%|██████████| 2/2 [00:00<00:00,  3.94it/s]

epoch 19: {'balanced_accuracy': 0.6755952380952381}


In [ ]:
best_model.push_to_hub("Lukee4/llama-letters", use_auth_token=True)


adapter_model.bin:   0%|          | 0.00/4.78M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Lukee4/biogpt-2019/commit/cd5b95f9e814257657e981e29d31c6ccb84809ea', commit_message='Upload model', commit_description='', oid='cd5b95f9e814257657e981e29d31c6ccb84809ea', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
for step, batch in enumerate(tqdm(test_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = best_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )
    metric2.add_batch(
        predictions=predictions,
        references=references,
    )


eval_metric = metric.compute()
eval_metric2 = metric2.compute()

print()
print()
print("results on the test data, after fine tunning, letters, llama:")
print(eval_metric)
print(eval_metric2)

100%|██████████| 2/2 [00:00<00:00,  2.89it/s]



results on the test data:
{'0': {'precision': 0.7037037037037037, 'recall': 0.7916666666666666, 'f1-score': 0.7450980392156864, 'support': 24}, '1': {'precision': 0.5454545454545454, 'recall': 0.42857142857142855, 'f1-score': 0.4799999999999999, 'support': 14}, 'accuracy': 0.6578947368421053, 'macro avg': {'precision': 0.6245791245791246, 'recall': 0.6101190476190476, 'f1-score': 0.6125490196078431, 'support': 38}, 'weighted avg': {'precision': 0.6454013822434875, 'recall': 0.6578947368421053, 'f1-score': 0.6474303405572757, 'support': 38}}
{'balanced_accuracy': 0.6101190476190476}
